### Full POP model works

In [1]:
include("../LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using DynamicPolynomials
using LinearAlgebra
using Dates
using HDF5
using TSSOS

Consider simple Lindblad master equation with just one dissipator:

$m
     \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+\gamma\left[J \rho J^\dagger - \frac{1}{2}\left\{ J^\dagger J, \rho \right\} \right]
$,

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} e_1 & h_1 - i h_2 \\ h_1 + i h_2 & 0
   \end{pmatrix}
$

 withot loss of generality we can possibly look for jump operator of the form:

$
J = \begin{pmatrix} a_{11} + i b_{11} & a_{12} \\ a_{21} + i b_{21} & -a_{11} - i b_{11} 
   \end{pmatrix} 
$

or with some additional assumptions

$
J = \begin{pmatrix} a_{11} & a_{12} + i b_{12}  \\ a_{21} + i b_{21} & -a_{11}
   \end{pmatrix} 
$

or assuming zero temperature 

$
J = \begin{pmatrix} a_{11} + i b_{11} & a_{12}   \\ 0 & -a_{11} - i b_{11}
   \end{pmatrix} 
$

In [2]:
@polyvar e[1]
@polyvar h[1:2]


Hˢʸᵐᵇ = [ e[1]               h[1] - im*h[2]
          h[1] + im*h[2]     0.             ]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 e₁                  h₁ + (0.0-1.0im)h₂
 h₁ + (0.0+1.0im)h₂  0.0+0.0im

In [3]:
method = "fix-Phi-up_"

@polyvar a[1:4]
@polyvar b[1:4]


Jˢʸᵐᵇ = [ a[1]+im*b[1]      a[2]
          a[3] + im*b[3]   -a[1]-im*b[1] ]


2×2 Matrix{Polynomial{true, Complex{Int64}}}:
 a₁ + (0+1im)b₁  a₂
 a₃ + (0+1im)b₃  -a₁ + (0-1im)b₁

In [4]:
data_dir = "../DATA/"
println(data_dir)

models_dir = "../MODELS/"
tests_dir = "../TESTS/"

dodeca_files = ["State_D"*string(n) for n=1:20];

basis_files = ["State_B"*string(n) for n=1:4];

../DATA/


In [5]:
all_files = vcat(dodeca_files, basis_files)
train_files = basis_files 
test_files = dodeca_files;

In [11]:
function scaling_poly(p::Polynomial)

    X = transpose(hcat([exponents(t) for t in terms(p)]...))

    # Get the scaling via linear regression
    scaling = X \ log.(abs.(coefficients(p)))

    exp.(abs.(scaling)) # 
end


function tssos_min(p)
    
    ################################################################################################
    #
    #   Try TSSOS on polynomial with scaled variables
    #
    ################################################################################################

    pd = p #/ maximum(abs.(coefficients(p)))

    # minimize
    # minimizer_scaled_tssos = nothing

    try
        opt,sol,data = tssos_first(pd, variables(pd), QUIET=true, solution=true, newton=true);
        previous_sol = sol

        while ~isnothing(sol)
            previous_sol = sol
            opt,sol,data = tssos_higher!(data; QUIET=true, solution=true);
        end

        global minimizer_tssos = previous_sol
    
    catch
        println("Scaled TSSOS failed")
        global minimizer_scaled_tssos = nothing

    end

    best_solution = variables(p) => minimizer_tssos

end

import Base.real

function coef_range(obj)
    maximum(abs.(coefficients(obj)))/minimum(abs.(coefficients(obj)))
end

using Optim
using Random

function local_rand_min(p)

    pd = p / maximum(abs.(coefficients(p)))

    # find variable scaling
    scale = scaling_poly(pd)

    # scale the polynomial
    p_scaled = subs(pd, variables(pd) => scale .* variables(pd))

    num_iterations = 100

    # Initialize the best minimizer and the minimum value
    best_minimizer = nothing
    best_min_value = Inf

    num_of_variables = length(variables(pd))

    for _ in 1:num_iterations
        # Generate a random initial condition
        initial_point = rand(num_of_variables).*250

        # Run local optimization
        result = Optim.optimize(p_scaled, initial_point, BFGS())
        #println(Optim.minimum(result))

        # Update the best minimizer if a better one is found
        if Optim.minimum(result) < best_min_value
            
            best_minimizer = Optim.minimizer(result)
            best_min_value = Optim.minimum(result)
            
        end

    end

    minimizer_scaled = scale .* best_minimizer

    solution = variables(p_scaled) => minimizer_scaled

end


local_rand_min (generic function with 1 method)

### Train models loop

In [15]:
date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))
models_file = "POP_LME_tssos_vs_random_"*method*date_and_time_string * ".h5"
tests_data_file_name = "POP_LME_tssos_vs_random_trn4_tst20_"*method*date_and_time_string * ".h5"

println(models_file)

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

for γᵢ in γ
    
    println("---------------")
    println("gamma = ", γᵢ)
    
    Hᴸᴹᴱ = [ 25.126        0
             0             0    ]
    γᶠ = parse(Float64, γᵢ)

    Jᴸᴹᴱ = [ 0    √(γᶠ)
             0     0. + 0im  ]

    objₑₓ = 0
    
    objₗₘₑ = 0
    
    for df_trn in train_files # loop over initial states

        ρᵗʳⁿ, tᵗʳⁿ = LiPoSID.get_rho_series(data_dir*df_trn*"_2CUT_data.h5", γᵢ)
        
        if length(tᵗʳⁿ) > 1200 end_train = 1200 else end_train = length(tᵗʳⁿ) end
            
        ρᵗʳⁿ = convert(Vector{Matrix{ComplexF64}}, ρᵗʳⁿ[1:end_train])
        
        ρᵗʳⁿₒₚₛ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρᵗʳⁿ]
        tᵗʳⁿ = convert(Vector{Float64}, tᵗʳⁿ[1:end_train])
    
        objₑₓ += LiPoSID.simpson_obj(ρᵗʳⁿ, tᵗʳⁿ,  Hˢʸᵐᵇ, [Jˢʸᵐᵇ])
         
        ρˡᵐᵉ = LiPoSID.Lindblad_time_evolution(basis, ρᵗʳⁿ[1], tᵗʳⁿ, Hᴸᴹᴱ, [Jᴸᴹᴱ])
        objₗₘₑ += LiPoSID.simpson_obj(ρˡᵐᵉ, tᵗʳⁿ,  Hˢʸᵐᵇ, [Jˢʸᵐᵇ])
            
        
    end # of files (initial states) loop
            
    coef_minₑₓ = minimum(abs.(coefficients(objₑₓ)))
    coef_maxₑₓ = maximum(abs.(coefficients(objₑₓ)))

    coefs_gapₑₓ = coef_maxₑₓ/coef_minₑₓ 
            
    coef_minₗₘₑ = minimum(abs.(coefficients(objₗₘₑ)))
    coef_maxₗₘₑ = maximum(abs.(coefficients(objₗₘₑ)))
            
    coefs_gapₗₘₑ= coef_maxₗₘₑ/coef_minₗₘₑ 
       
    println("-------------------------------------------")
    println(" POP on ansatz LME simulation")
            
    #objₗₘₑrm = coefs_gapₗₘₑ > 1e6 ? remove_micro_coefs(objₗₘₑ, cutoffₗₘₑ) : objₗₘₑ
    
    solₗₘₑtssos  = tssos_min(objₗₘₑ)
    solₗₘₑrnd  = local_rand_min(objₗₘₑ)
    
    println("-------------------------------------------")
    
    solₗₘₑ = [e[1], h[1], h[2], a[1], a[2], a[3], b[1], b[3]] => [25.126, 0, 0, 0,  √(γᶠ), 0, 0, 0]
    
    obj_val_lme = convert(Float64, subs(objₗₘₑ, solₗₘₑ))
    
    obj_val_sid_on_lme = convert(Float64, subs(objₗₘₑ, solₗₘₑrnd))
    
    if obj_val_sid_on_lme < obj_val_lme
        println(" simulated LME random local overfitting")
    end
    
    Hˢⁱᵈₗₘₑtssos = subs(Hˢʸᵐᵇ, solₗₘₑtssos)
    Jˢⁱᵈₗₘₑtssos = subs(Jˢʸᵐᵇ, solₗₘₑtssos)

    Hˢⁱᵈₗₘₑrnd = subs(Hˢʸᵐᵇ, solₗₘₑrnd)
    Jˢⁱᵈₗₘₑrnd = subs(Jˢʸᵐᵇ, solₗₘₑrnd)
      
    println(" POP on ansatz EXACT Kurt data ")
                
    solₑₓtssos = tssos_min(objₑₓ)
    solₑₓrnd = local_rand_min(objₑₓ)
   
    obj_val_lme_on_exact = convert(Float64, subs(objₑₓ, solₗₘₑrnd))
    obj_val_sid_on_exact = convert(Float64, subs(objₑₓ, solₑₓrnd))
    
    if obj_val_sid_on_exact < obj_val_lme_on_exact
        println(" Exact data random local overfitting")
    end

    Hˢⁱᵈₑₓtssos = DynamicPolynomials.subs(Hˢʸᵐᵇ, solₑₓtssos)
    Jˢⁱᵈₑₓtssos = DynamicPolynomials.subs(Jˢʸᵐᵇ, solₑₓtssos)

    Hˢⁱᵈₑₓrnd = DynamicPolynomials.subs(Hˢʸᵐᵇ, solₑₓrnd)
    Jˢⁱᵈₑₓrnd = DynamicPolynomials.subs(Jˢʸᵐᵇ, solₑₓrnd)
    
   
    h5open(models_dir*models_file,"cw") do fid  # read-write, create file if not existing, preserve existing contents

        γ_group = create_group(fid, "gamma_"*γᵢ)
        
        
        γ_group["coef_min_ex"] = coef_minₑₓ
        γ_group["coef_max_ex"] = coef_maxₑₓ
        γ_group["coef_gap_ex"] = coefs_gapₑₓ
                        
        γ_group["coef_min_lme"] = coef_minₗₘₑ
        γ_group["coef_max_lme"] = coef_maxₗₘₑ
        γ_group["coef_gap_lme"] = coefs_gapₗₘₑ       
                        
        γ_group["obj_val_lme"] = obj_val_lme
        γ_group["obj_val_sid_on_lme"] = obj_val_sid_on_lme
        
        γ_group["obj_val_lme_on_exact"] = obj_val_lme_on_exact
        γ_group["obj_val_sid_on_exact"] = obj_val_sid_on_exact


        γ_group["H_sid_sb_tssos"] = convert.(ComplexF64, Hˢⁱᵈₑₓtssos)
        γ_group["A_sid_sb_tssos"] = convert.(ComplexF64, Jˢⁱᵈₑₓtssos)

        γ_group["H_sid_sb_rnd"] = convert.(ComplexF64, Hˢⁱᵈₑₓrnd)
        γ_group["A_sid_sb_rnd"] = convert.(ComplexF64, Jˢⁱᵈₑₓrnd)
        
        γ_group["H_lme"] = convert.(ComplexF64, Hᴸᴹᴱ)
        γ_group["A_lme"] = convert.(ComplexF64, Jᴸᴹᴱ)
        
        γ_group["H_sid_lme_tssos"] = convert.(ComplexF64, Hˢⁱᵈₗₘₑtssos)
        γ_group["A_sid_lme_tssos"] = convert.(ComplexF64, Jˢⁱᵈₗₘₑtssos)

        γ_group["H_sid_lme_rnd"] = convert.(ComplexF64, Hˢⁱᵈₗₘₑrnd)
        γ_group["A_sid_lme_rnd"] = convert.(ComplexF64, Jˢⁱᵈₗₘₑrnd)

        
    end # of HDF5 writing
    
    
    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ)
    end 

    println("STATE    F_LME_vs_SB    F_rnd_vs_LME    F_rnd_vs_sb    F_tssos_vs_LME    F_tssos_vs_SB")
    
    for df in test_files # loop over initial states
        
        print(df*" ")

        start_time = time()

        ρₛ, tₛ = LiPoSID.get_rho_series(data_dir*df*"_2CUT_data.h5", γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)

        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]

        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        tᴸᴹᴱ, ρᴸᴹᴱ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis,Hᴸᴹᴱ), [DenseOperator(basis,Jᴸᴹᴱ)])

        tˢⁱᵈₗₘrnd, ρˢⁱᵈₗₘₑrnd  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈₗₘₑrnd), [DenseOperator(basis,Jˢⁱᵈₗₘₑrnd)])
        tˢⁱᵈₑₓrnd, ρˢⁱᵈₑₓrnd  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈₑₓrnd), [DenseOperator(basis,Jˢⁱᵈₑₓrnd)])   

        tˢⁱᵈₗₘtssos, ρˢⁱᵈₗₘₑtssos  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈₗₘₑtssos), [DenseOperator(basis,Jˢⁱᵈₗₘₑtssos)])
        tˢⁱᵈₑₓtssos, ρˢⁱᵈₑₓtssos  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈₑₓtssos), [DenseOperator(basis,Jˢⁱᵈₑₓtssos)])  
        

        Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρᴸᴹᴱ)]

        Fˢⁱᵈᴸᴹᴱₗₘₑrnd = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρˢⁱᵈₗₘₑrnd, ρᴸᴹᴱ)]
        Fˢⁱᵈᵉˣₑₓrnd = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρˢⁱᵈₑₓrnd, ρᵗˢᵗ)]

        Fˢⁱᵈᴸᴹᴱₗₘₑtssos = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρˢⁱᵈₗₘₑtssos, ρᴸᴹᴱ)]
        Fˢⁱᵈᵉˣₑₓtssos = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρˢⁱᵈₑₓtssos, ρᵗˢᵗ)]
                    

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group

            init_state_group = create_group(γ_group, df) # create initial state group
            init_state_group["F_lme_exact"] = convert.(Float64, Fᴸᴹᴱₑₓ)

            init_state_group["F_sidlme_lme_tssos"] = convert.(Float64, Fˢⁱᵈᴸᴹᴱₗₘₑtssos)
            init_state_group["F_sidexact_exact_tssos"] = convert.(Float64, Fˢⁱᵈᵉˣₑₓtssos)

            init_state_group["F_sidlme_lme_rnd"] = convert.(Float64, Fˢⁱᵈᴸᴹᴱₗₘₑrnd)
            init_state_group["F_sidexact_exact_rnd"] = convert.(Float64, Fˢⁱᵈᵉˣₑₓrnd)
            
        end
                    
        print(minimum(Fᴸᴹᴱₑₓ), minimum(Fˢⁱᵈᴸᴹᴱₗₘₑrnd), minimum(Fˢⁱᵈᵉˣₑₓrnd), minimum(Fˢⁱᵈᴸᴹᴱₗₘₑtssos), minimum(Fˢⁱᵈᵉˣₑₓtssos))
        println(" ")
    
    end
                
    println()
    println()
                            
end

POP_LME_tssos_vs_random_fix-Phi-up_2024-Jan-09_at_14-34.h5
---------------
gamma = 0.079477
-------------------------------------------


 POP on ansatz LME simulation
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 9.502233422981925e-8
Found a locally optimal solution by Ipopt, giving an upper bound: 0.01048907.
The relative optimality gap is: 1.048897%.
No higher TS step of the TSSOS hierarchy!


-------------------------------------------
 simulated LME random local overfitting
 POP on ansatz EXACT Kurt data 
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 0.0005153423032621579
Global optimality certified with relative optimality gap 0.000001%!
No higher TS step of the TSSOS hierarchy!
 Exact data random local overfitting


STATE    F_LME_vs_SB    F_rnd_vs_LME    F_rnd_vs_sb    F_tssos_vs_LME    F_tssos_vs_SB


State_D1 

...

0.99999999999999540.99721346868261210.99999999999999530.9972134687124105 
State_D13 0.73688039059027650.9999999999999940.99771126402233650.9999999999999940.9977112640437946 
State_D14 0.7368897118012756

0.99999999999999430.99731818403253760.99999999999999470.9973181840578893 
State_D15 0.73672455527660240.99999999999999530.99592558963892850.99999999999999560.9959255896821393 
State_D16 0.7367245552785794

0.99999999999999610.99634048764404470.99999999999999560.9963404876825823 
State_D17 0.73977887157886220.99999999999999170.99576568428940240.99999999999999270.9957656842036298 
State_D18 0.7398143078838488

0.99999999999999360.99899985599020140.99999999999999370.998999856000152 
State_D19 0.72932820338411670.99999999999999580.99667743391353760.99999999999999560.9966774339503194 
State_D20 0.7343569139368477

0.99999999999999470.99686523849058850.99999999999999460.9968652385201188 


